# Cassandra Database for "Sparkify" User Data

This data engineering project leverages Apache Cassandra to create and populate a distributed database for the storage of user data for a hypothetical the music listening app called Sparkify. Three queries will be made of the data:

1) The artists, songs, and song length made in a particular session (session_id = 338)

2) The artist, song name, and user (first and lanst names) for a particular user_id and session (session_id = 182)

3) The first and last user names of every listener who listened to a particular song ("All Hands Against His Own")

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")


#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

# Part II. Complete the Apache Cassandra coding.

Now that the data has been aggregated into a single denormalized .csv file ('event_datafile_new.csv') individual Cassandra tables can be created for each query.

# Query 1) The artists, songs, and song length made in a particular session (session_id = 338)

I selected session_id, item_in_session as my Primary Key becuase they would be necissary 
components of the WHERE conditional.

In [102]:
# Creating Table: song_data

session.execute("CREATE TABLE song_data(session_id int, item_in_session int, artist text, song_title text, length float, PRIMARY KEY(session_Id, item_in_session))")  




In [88]:
# Populating the table song_data 
# Note all data from .csv is formated as datatype=string 
# must be converted
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        length = float(line[5]) # Converting line[5] (type=string) to a float
        session_id = int(line[8]) # Converting line[8] (type=string) to an integer
        item_in_session = int(line[3]) # Converting line[3] (type=string) to an integer
        query = "INSERT INTO song_data(session_id, item_in_session, artist, song_title, length) "
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (session_id,item_in_session,line[0], line[9],length))

In [99]:
# Selecting artist name, song title, and song length for session_id = 338
rows = session.execute("SELECT artist, song_title, length FROM song_data WHERE session_id = 338 AND item_in_session = 4")

print('The artist, song title, and song length heard during sessionId = 338, and item_in_session = 4 are:')
for row in rows:
    print (row.artist, row.song_title, row.length)

The artist, song title, and song length heard during sessionId = 338, and item_in_session = 4 are:
Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query 2) The artist, song name, and user (first and last names) for a particular user_id and session (session_id = 182)

I selected both user_id and session_id as my Composite Primary Key to use in the WHERE conditional. I selected item_number as my Clustering Column in order to 'sort' the different songs in ascending order. 

In [90]:
# Creating Table: user_artists

session.execute("CREATE TABLE user_artists(user_id int, session_id int, item_in_session int,\
artist text, song_title text, first_name text, last_name text,\
PRIMARY KEY((user_id, session_id), item_in_session))")


In [91]:
# Populating the table user_artists
# Note all data from .csv is formated as datatype=string 
# must be converted

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        user_id = int(line[10]) # converting line[10] (type=string) to an integer
        session_id = int(line[8]) # Converting line[8] (type=string) to an integer
        item_in_session = int(line[3]) # Converting line[3] (type=string) to an integer
        query = "INSERT INTO user_artists(user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (user_id, session_id, item_in_session, line[0], line[9], line[1], line[4]))

In [92]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute("SELECT artist, song_title, first_name, last_name FROM user_artists WHERE user_id = 10 AND session_id = 182 ORDER BY item_in_session ASC")

print("The names of artists, songs, and username for user_id: 10, session_id 182 are: ")
for row in rows:
    print (row.artist, row.song_title, row.first_name, row.last_name)

The names of artists, songs, and username for user_id: 10, session_id 182 are: 
Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Query 3) The first and last user names of every listener who listened to a particular song ("All Hands Against His Own")

I selected song_title and user_id as a Composite Primary Key in order to utilize the WHERE conditional on song_title while reteriving the names of unique users. 

In [96]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
session.execute("CREATE TABLE all_hands_fans(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY(song_title, user_id)")

                    

                    

In [103]:
# Populating Table all_hands_fans
# Note all data from .csv is formated as datatype=string 
# must be converted

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        user_id = int(line[10]) # Converting line[10] (type=string) to an integer
        query = "INSERT INTO all_hands_fans(song_title, user_id, first_name, last_name) "
        query = query + "VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9], user_id, line[1],line[4]))

In [98]:
# Selecting the user name (first and last) of everyone who listened to the song 'All Hands Against His Own'
rows = session.execute("SELECT first_name, last_name FROM all_hands_fans WHERE song_title = 'All Hands Against His Own'")

print("The full name of every user who listened to 'All Hands Against His Own': ")
for row in rows:
    print (row.first_name, row.last_name)

The full name of every user who listened to 'All Hands Against His Own': 
Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [81]:
session.execute('DROP TABLE IF EXISTS song_data')
session.execute('DROP TABLE IF EXISTS user_artists')
session.execute('DROP TABLE IF EXISTS all_hands_fans')

### Close the session and cluster connection¶

In [66]:
session.shutdown()
cluster.shutdown()